In [1]:
import re
def read_file():
    T = {}
    M = {}
    cannot_be_together = []
    must_have = []
    t = 1
    with open("input-data.txt") as file:
        for transaction in file:
            transList = []
            transarr = re.findall(r'\{(.*?)\}',transaction)
            str1 = ''.join(transarr)
            T[int(t)] = [x.strip() for x in str1.split(',')]
            t = t + 1
        
    with open("parameter-file.txt") as param:
        param_data = [line for line in param]
    for d in param_data:
        if "MIS" in d:
            match = re.match(r'^.*\((.*)\).*= (\d*\.\d*)', d)
            M[match.group(1)] = float(match.group(2))
        elif "SDC" in d:
            max_sup_diff = float(re.match(r'.*= (.*)', d).group(1))
        elif "must-have" in d:
            must_have = [x.strip() for x in d.split(':')[1].split('or')]
        elif "cannot_be_together" in d:
            cannot_be_together = re.findall(r'{.*?}',d)
    return T,M,max_sup_diff,cannot_be_together,must_have

In [2]:
##final
from operator import itemgetter
from collections import OrderedDict
def sortOnMIS(M):
    #sorted_m = sorted(M.items(), key=operator.itemgetter(1))
    sorted_m = sorted(M.items(), key=itemgetter(1))
    return sorted_m

MIS(10) = 0.43
MIS(20) = 0.30
MIS(30) = 0.30
MIS(40) = 0.40
MIS(50) = 0.40
MIS(60) = 0.30
MIS(70) = 0.20
MIS(80) = 0.20
MIS(90) = 0.20
MIS(100) = 0.10
MIS(120) = 0.20
MIS(140) = 0.15

In [3]:
#M = {"10":0.43,"20":0.30,"30":0.30,"40":0.40,"50":0.40,"60":0.30,"70":0.20,"80":0.20,"90":0.20,"100":0.10,"120":0.20,"140":0.15}
#M

{'10': 0.43,
 '100': 0.1,
 '120': 0.2,
 '140': 0.15,
 '20': 0.3,
 '30': 0.3,
 '40': 0.4,
 '50': 0.4,
 '60': 0.3,
 '70': 0.2,
 '80': 0.2,
 '90': 0.2}

In [4]:
#sorted_m = sortOnMIS(M)
#sorted_m


[('100', 0.1),
 ('140', 0.15),
 ('70', 0.2),
 ('80', 0.2),
 ('90', 0.2),
 ('120', 0.2),
 ('20', 0.3),
 ('30', 0.3),
 ('60', 0.3),
 ('40', 0.4),
 ('50', 0.4),
 ('10', 0.43)]

In [5]:
## accessing values from tuples
##result = [m[1] for m in sorted_m if m[0] == '100' ]
##result

[0.1]

{1: ['{20,', '30,', '80,', '70,', '50,', '90}'], 2: ['{20,', '10,', '80,', '70}'], 3: ['{10,', '20,', '80}'], 4: ['{20,', '30,', '80}'], 5: ['{20,', '80}'], 6: ['{20,', '30,', '80,', '70,', '50,', '90,', '100,', '120,', '140}']}

In [6]:
#T={1:{'20', '30', '80', '70', '50', '90'},2:{'20', '10', '80', '70'},3:{'10', '20', '80'},4:{'20', '30', '80'},5:{'20', '80'},6:{'20', '30', '80', '70', '50', '90', '100', '120', '140'}}

In [7]:
#T

{1: {'20', '30', '50', '70', '80', '90'},
 2: {'10', '20', '70', '80'},
 3: {'10', '20', '80'},
 4: {'20', '30', '80'},
 5: {'20', '80'},
 6: {'100', '120', '140', '20', '30', '50', '70', '80', '90'}}

In [3]:
##final
def init_pass(T,sorted_m):
    L = []
    supp = calculate_support(T,sorted_m)
    first_sorted_m = sorted_m[0]
    len_t = len(T)
    L.append(first_sorted_m[0])
    mis_i = first_sorted_m[1]
    for m in sorted_m:
        if first_sorted_m[0] not in m:
            if supp[m[0]] >= mis_i:
                L.append(m[0])
    return L,supp

In [4]:
##final
def calculate_support(T,sorted_m):
    supp={}
    len_t = len(T)
    for m in sorted_m:
        count=0
        for t in T.values():
            if m[0] in t:
                count +=1
        supp[m[0]] = count/len_t
    return supp

L,supp = init_pass(T,sorted_m)
F = []
#change this
must_have=['20','40','50']
# F = calculate_F1(L,sorted_m,supp,must_have)
print(supp)
print(F)

In [11]:
def calculate_F1(L,sorted_m,supp,must_have,F):
    F1=[]
    for l in L:
        mis_l = [m[1] for m in sorted_m if m[0] == l]
        if supp[l] >= mis_l[0]:
            if l in must_have:
                F1.append(l)
    F.append(F1)
    return F

In [16]:
## sorted_m - stores mis in sorted order
## T - Dictionary of transactions
## supp - Dictionary for all supports of elements in m
## L - List of L
## F1 - List of Frequent 1
def ms_apriori():
    T,M,max_sup_diff,cannot_be_together,must_have = read_file()
    sorted_m = sortOnMIS(M)
    L,supp = init_pass(T,sorted_m)
    F=[]
    F = calculate_F1(L,sorted_m,supp,must_have,F)
    print(F)
    k = 2 
    
    

In [17]:
ms_apriori()

[['20']]
